In [1]:
!pip install selenium
!pip install webdriver

ERROR: Could not find a version that satisfies the requirement webdriver (from versions: none)
ERROR: No matching distribution found for webdriver


In [2]:
import csv 
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import os
import sys
os.path.dirname(sys.executable)
from time import sleep
from selenium import webdriver
from selenium.webdriver import Chrome 
from selenium.webdriver.chrome.options import Options as ChromeOptions
browser = webdriver.Chrome()
driver = webdriver.Chrome()

In [3]:
def get_tweet_data(card):
    """Extract data from tweet card"""
    username = card.find_element_by_xpath('.//span').text
    try:
        handle = card.find_element_by_xpath('.//span[contains(text(), "@")]').text
    except NoSuchElementException:
        return
    
    try:
        postdate = card.find_element_by_xpath('.//time').get_attribute('datetime')
    except NoSuchElementException:
        return
    
    comment = card.find_element_by_xpath('.//div[2]/div[2]/div[1]').text
    responding = card.find_element_by_xpath('.//div[2]/div[2]/div[2]').text
    text = comment + responding
    reply_cnt = card.find_element_by_xpath('.//div[@data-testid="reply"]').text
    retweet_cnt = card.find_element_by_xpath('.//div[@data-testid="retweet"]').text
    like_cnt = card.find_element_by_xpath('.//div[@data-testid="like"]').text

    
    tweet = (username, handle, postdate, text, reply_cnt, retweet_cnt, like_cnt)
    return tweet  

In [4]:
#log in to twitter
driver.get('https://twitter.com/login')

In [5]:
search_term = input('search term: ')

#driver.get('https://twitter.com/search')
driver.maximize_window()
sleep(5)

# find search input and search for term
search_input = driver.find_element_by_xpath('//input[@aria-label="Search query"]')
search_input.send_keys(search_term)
search_input.send_keys(Keys.RETURN)
sleep(1)

# navigate to historical 'latest' tab
driver.find_element_by_link_text('Latest').click()

search term: 蔣萬安


In [ ]:
# get all tweets on the page
data = []
tweet_ids = set()
last_position = driver.execute_script("return window.pageYOffset;")
scrolling = True

while scrolling:
    page_cards = driver.find_elements_by_xpath('//article[@data-testid="tweet"]')
    for card in page_cards[-15:]:
        tweet = get_tweet_data(card)
        if tweet:
            tweet_id = ''.join(tweet)
            if tweet_id not in tweet_ids:
                tweet_ids.add(tweet_id)
                data.append(tweet)
            
    scroll_attempt = 0
    while True:
        # check scroll position
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        sleep(2)
        curr_position = driver.execute_script("return window.pageYOffset;")
        if last_position == curr_position:
            scroll_attempt += 1
            
            # end of scroll region
            if scroll_attempt >= 3:
                scrolling = False
                break
            else:
                sleep(2) # attempt another scroll
        else:
            last_position = curr_position
            break

# close the web driver
driver.close()

In [8]:
data

[('三立iNEWS',
  '@inewsknowledge',
  '2022-12-10T05:00:01.000Z',
  '三立iNEWS\n@inewsknowledge\n·\n20m于北辰曝"高虹安對柯P超不爽！" 虹辦助理繳公積金 當月應退款竟比薪水還高！獲頒當選證書卻不開心？于：虹一肚子火氣 柯害她惹一身腥｜https://youtu.be/bPaWM7scyTM\n2',
  '',
  '',
  '2'),
 ('三戒',
  '@xuyuinfo5',
  '2022-12-10T04:43:00.000Z',
  '三戒\n@xuyuinfo5\n·\n37mReplying to \n@al6wul4wul4\n关于高虹安未来动向的几点猜测，\n①放弃就任新竹市长，继续担任立委，用好用足国会议员这把保护伞\n②畏罪潜逃，很可能逃去中国\n③检方可能25日凌晨拘押高虹安',
  '',
  '',
  ''),
 ('高詩特',
  '@hqvSpv2SoXIj4Zn',
  '2022-12-10T04:34:39.000Z',
  '高詩特\n@hqvSpv2SoXIj4Zn\n·\n46m蔣萬安留首長搶輸高虹安？ 柯文哲笑：我的局處長太優秀\n柯文哲將帥無能，累死三軍，連累主官．\n市政最後一名，實至名歸．\n11',
  '',
  '',
  '11'),
 ('Kuán Kui-Tîng',
  '@guanguiting',
  '2022-12-10T04:06:18.000Z',
  'Kuán Kui-Tîng\n@guanguiting\n·\n1hReplying to \n@NotNand\n會使，棄職著無就職後立委自動辭職兮問題。不過立法此會期只到月底，臨時會盡拖也是1/13。高虹安準是三更無死，也袂拖過五更。\n3',
  '',
  '',
  '3'),
 ('三立LIVE新聞',
  '@setlivenews',
  '2022-12-10T04:00:01.000Z',
  '三立LIVE新聞\n@setlivenews\n·\n1h周玉蔻預言"一辭立委就被收押"! 高虹安否認國庫通黨庫 強調江青山跟拍柯文哲純屬"個人追星"行為? 小內閣名單遲未公布 高虹安:多位人選洽談中｜https://you

In [9]:
with open('scrap_GaoHongan.csv', 'w', newline='', encoding='utf-8') as f:
    header = ['UserName', 'Handle', 'Timestamp', 'Text', 'Comments', 'Likes', 'Retweets']
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(data)